# Tyche Example

## Set up.

### One only needs to execute the following line once, in order to make sure recent enough packages are installed.

In [ ]:
!pip install 'numpy>=1.17.2' 'pandas>=0.25.1'

### Import packages.

In [1]:
import numpy             as np
import matplotlib.pyplot as pl
import pandas            as pd
import re                as re
import scipy.stats       as st

# The `tyche` package is located at <https://github.com/NREL/portfolio/tree/master/production-function/framework/code/tyche/>.
import tyche             as ty

from copy import deepcopy

## Load data.

### The data are stored in a set of tab-separated value files in a folder.

In [2]:
designs = ty.Designs("../data")

In [3]:
investments = ty.Investments("../data")

### Compile the production and metric functions for each technology in the dataset.

In [20]:
designs.compile()

## Examine the data.

### The `functions` table specifies where the Python code for each technology resides.

In [21]:
designs.functions

,Style,Module,Capital,Fixed,Production,Metrics,Notes
Technology,,,,,,,
Simple electrolysis,numpy,simple_electrolysis,capital_cost,fixed_cost,production,metrics,


Right now, only the style `numpy` is supported.

### The `indices` table defines the subscripts for variables.

In [22]:
designs.indices

Offset  Description Notes
Technology          Type    Index                                 
Simple electrolysis Capital Catalyst          0     Catalyst      
                    Fixed   Rent              0         Rent      
                    Input   Electricity       1  Electricity      
                            Water             0        Water      
                    Metric  Jobs              0         Jobs      
                    Output  Hydrogen          1     Hydrogen      
                            Oxygen            0       Oxygen

### The `designs` table contains the cost, input, efficiency, and price data for a scenario.

In [23]:
designs.designs

Value  \
Technology          Scenario                          Variable          Index                                                       
Simple electrolysis Base Electrolysis                 Input             Electricity                                           279   
                                                                        Water                                               19.04   
                                                      Input efficiency  Electricity                                          0.85   
                                                                        Water                                                0.95   
                                                      Input price       Electricity                                       3.33e-5   
                                                                        Water                                              4.8e-3   
                                                      Lifetime          Catalyst                                                3   
                                                      Output efficiency Hydrogen                                             0.90   
                                                                        Oxygen                                               0.90   
                                                      Output price      Hydrogen                                           1.0e-2   
                                                                        Oxygen                                             3.0e-3   
                                                      Scale             NaN                                                  6650   
                    Fast Progress on Electrolysis     Input             Electricity                                           279   
                                                                        Water                                               19.04   
                                                      Input efficiency  Electricity  st.truncnorm(-3, 0.75, loc=0.97, scale=0.04)   
                                                                        Water          st.truncnorm(-3, -2, loc=0.97, scale=0.01)   
                                                      Input price       Electricity                                       3.33e-5   
                                                                        Water                                              4.8e-3   
                                                      Lifetime          Catalyst                                                3   
                                                      Output efficiency Hydrogen              st.beta(3, 2, loc=0.90, scale=0.03)   
                                                                        Oxygen                st.beta(3, 2, loc=0.90, scale=0.06)   
                                                      Output price      Hydrogen                                           1.0e-2   
                                                                        Oxygen                                             3.0e-3   
                                                      Scale             NaN                                                  6650   
                    Moderate Progress on Electrolysis Input             Electricity                                           279   
                                                                        Water                                               19.04   
                                                      Input efficiency  Electricity  st.truncnorm(-2, 1.75, loc=0.93, scale=0.04)   
                                                                        Water          st.truncnorm(-2, -3, loc=0.97, scale=0.01)   
                                                      Input price       Electricity                                       3.33e-5   
 

### The `parameters` table contains additional techno-economic parameters for each technology.

In [24]:
designs.parameters

Offset  \
Technology          Scenario                          Parameter                                     
Simple electrolysis Base Electrolysis                 Electricity consumption                   3   
                                                      Hydrogen production                       1   
                                                      Jobs                                      4   
                                                      Oxygen production                         0   
                                                      Reference capital cost for catalyst       6   
                                                      Reference fixed cost for rent             7   
                                                      Reference scale                           5   
                                                      Water consumption                         2   
                    Fast Progress on Electrolysis     Electricity consumption                   3   
                                                      Hydrogen production                       1   
                                                      Jobs                                      4   
                                                      Oxygen production                         0   
                                                      Reference capital cost for catalyst       6   
                                                      Reference fixed cost for rent             7   
                                                      Reference scale                           5   
                                                      Water consumption                         2   
                    Moderate Progress on Electrolysis Electricity consumption                   3   
                                                      Hydrogen production                       1   
                                                      Jobs                                      4   
                                                      Oxygen production                         0   
                                                      Reference capital cost for catalyst       6   
                                                      Reference fixed cost for rent             7   
                                                      Reference scale                           5   
                                                      Water consumption                         2   
                    Slow Progress on Electrolysis     Electricity consumption                   3   
                                                      Hydrogen production                       1   
                                                      Jobs                                      4   
                                                      Oxygen production                         0   
                                                      Reference capital cost for catalyst       6   
                                                      Reference fixed cost for rent             7   
                                                      Reference scale                           5   
                                                      Water consumption                         2   

                                                                                            Value  \
Technology          Scenario                          Parameter                                     
Simple electrolysis Base Electrolysis                 Electricity consumption                 237   
                                                      Hydrogen production                    2.00   
                                                      Jobs                                 1.5e-4   
                                                      Oxygen production                     16.00   
                                                  

### The `results` table specifies the units of measure for results of computations.

In [25]:
designs.results

Units Notes
Technology          Variable Index                   
Simple electrolysis Cost     Cost      USD/mole      
                    Metric   Jobs      job/mole      
                    Output   Hydrogen    g/mole      
                             Oxygen      g/mole

### The `tranches` table specifies multually exclusive possibilities for investments: only one `Tranch` may be selected for each `Cateogry`.

In [26]:
investments.tranches

Notes
Category         Tranche                 Scenario                               
Electrolysis R&D High Electrolysis R&D   Fast Progress on Electrolysis          
                 Low Electrolysis R&D    Slow Progress on Electrolysis          
                 Medium Electrolysis R&D Moderate Progress on Electrolysis      
                 No Electrolysis R&D     Base Electrolysis

### The `investments` table bundles a consistent set of tranches (one per category) into an overall investment.

In [27]:
investments.investments

,,,Amount,Notes
Investment,Category,Tranche,,
High R&D Spending,Electrolysis R&D,High Electrolysis R&D,5000000.0,
Low R&D Spending,Electrolysis R&D,Low Electrolysis R&D,1000000.0,
Medium R&D Spending,Electrolysis R&D,Medium Electrolysis R&D,2500000.0,
No R&D Spending,Electrolysis R&D,No Electrolysis R&D,0.0,


## Evaluate the scenarios in the dataset.

In [ ]:
scenario_results = designs.evaluate_scenarios()

In [ ]:
scenario_results

## Evaluate the investments in the dataset.

In [ ]:
investment_results = investments.evaluate_investments(designs)

### Costs of investments.

In [ ]:
investment_results.amounts

### Benefits of investments.

In [ ]:
investment_results.metrics

In [ ]:
investment_results.summary

## Sensitity analysis.

### Vary the four efficiencies in the design.

In [ ]:
# Four variables are involved.
variables = [
    ("Input efficiency" , "Water"      ),
    ("Input efficiency" , "Electricity"),
    ("Output efficiency", "Oxygen"     ),
    ("Output efficiency", "Hydrogen"   ),
]

In [ ]:
# Let efficiencies range from 0.75 to 0.975.
efficiencies = np.arange(0.750, 1.000, 0.025)
efficiencies

### Start from the base case.

In [ ]:
base_design = designs.designs.xs("Base Electrolysis", level=1, drop_level=False)
base_design

In [ ]:
base_parameters = designs.parameters.xs("Base Electrolysis", level=1, drop_level=False)
base_parameters

### Generate the new scenarios and append them to the previous ones.

In [ ]:
sensitivities = deepcopy(designs)
sensitivities.designs = sensitivities.designs[0:0]
sensitivities.parameters = sensitivities.parameters[0:0]

In [ ]:
# Iterate over variables and efficiencies.
for variable, index in variables:
    for efficiency in efficiencies:

        # Name the scenario.
        scenario = "Let " + variable + " @ " + index + " = " + str(round(efficiency, 3))
        
        # Alter the base case.
        vary_design = base_design.rename(index={"Base Electrolysis" : scenario}, level=1)
        vary_design.loc[("Simple electrolysis", scenario, variable, index), "Value"] = efficiency
        
        # Keep the parameters the same.
        vary_parameters = base_parameters.rename(index={"Base Electrolysis" : scenario}, level=1)
        
        # Append the results to the existing table of scenarios.
        sensitivities.designs = sensitivities.designs.append(vary_design)
        sensitivities.parameters = sensitivities.parameters.append(vary_parameters)

#### See how many rows there are in the tables now.

In [ ]:
sensitivities.designs.shape

In [ ]:
sensitivities.parameters.shape

### Compute the results.

In [ ]:
results = sensitivities.evaluate_scenarios()
results

### Plot the cost results.

In [ ]:
cost_results = results.xs("Cost", level=3).reset_index()[["Scenario", "Value"]]

In [ ]:
cost_results["Variable"  ] = cost_results["Scenario"].apply(lambda x: re.sub(r'^Let (.*) @ (.*) =.*$', '\\1[\\2]', x))
cost_results["Efficiency"] = cost_results["Scenario"].apply(lambda x: float(re.sub(r'^.*= (.*)$', '\\1', x)))
cost_results["Cost [USD/mole]"] = cost_results["Value"]

In [ ]:
cost_results = cost_results[["Variable", "Efficiency", "Cost [USD/mole]"]]
cost_results[1:10]

In [ ]:
# Here is a really simple plot.
cost_results.plot(
    x="Efficiency",
    y="Cost [USD/mole]",
    c=cost_results["Variable"].apply(lambda v: {
        "Input efficiency[Water]"       : "blue"  ,
        "Input efficiency[Electricity]" : "orange",
        "Output efficiency[Oxygen]"     : "green" ,
        "Output efficiency[Hydrogen]"   : "red"   ,
    }[v]),
    kind="scatter"
)